In [4]:
from pathlib import Path
import duckdb
import pandas as pd

data_root = Path("D:\\데이터\\건축데이터 건축허브 개방데이터")
path_data = data_root / "기본개요_표제부_층별개요_조인.parquet"

In [5]:
# Open a DuckDB connection
con = duckdb.connect()

joined_db = con.sql(f"SELECT * FROM read_parquet('{path_data.as_posix()}')")

# Show columns of joined_db
print(joined_db.limit(5).df().columns)
print()

count = con.execute("SELECT COUNT(*) FROM joined_db").fetchone()[0]  # type: ignore
print(count)
print()

# show heads of the tables
print(f"Head of {path_data.stem}:")
df = con.execute("SELECT * FROM joined_db LIMIT 5").fetchdf()
print(df)
print()

Index(['관리_건축물대장_PK', '관리_상위_건축물대장_PK', '대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드',
       '대장_종류_코드_명', '대지_위치', '도로명_대지_위치', '건물_명', '시군구_코드', '법정동_코드',
       '대지_구분_코드', '번', '지', '특수지_명', '블록', '로트', '외필지_수', '새주소_도로_코드',
       '새주소_법정동_코드', '새주소_지상지하_코드', '새주소_본_번', '새주소_부_번', '지역_코드', '지구_코드',
       '구역_코드', '지역_코드_명', '지구_코드_명', '구역_코드_명', '생성_일자', '동_명', '주_부속_구분_코드',
       '주_부속_구분_코드_명', '대지_면적(㎡)', '건축_면적(㎡)', '연면적(㎡)', '용적_률_산정_연면적(㎡)',
       '구조_코드', '주_용도_코드', '지붕_코드', '세대_수(세대)', '가구_수(가구)', '높이(m)', '지상_층_수',
       '지하_층_수', '부속_건축물_수', '부속_건축물_면적(㎡)', '총_동_연면적(㎡)', '허가_일', '착공_일',
       '사용승인_일', '허가번호_년', '호_수(호)', '생성_일자_1', '층_구분_코드', '층_구분_코드_명', '층_번호',
       '층_번호_명', '층_구조_코드', '층_주_용도_코드', '층_주_용도_코드_명', '층_면적', '층_주_부속_구분_코드',
       '층_면적_제외_여부'],
      dtype='object')

19614969

Head of 기본개요_표제부_층별개요_조인:
  관리_건축물대장_PK 관리_상위_건축물대장_PK 대장_구분_코드 대장_구분_코드_명 대장_종류_코드 대장_종류_코드_명  \
0  1016119901           None        2         집합        3        표제부   
1  1016119

In [6]:
query = """SELECT
    "관리_건축물대장_PK",
    "시군구_코드",
    "법정동_코드",
    "시군구_코드" || "법정동_코드" AS long_법정동_코드,
    "지역_코드",
    "지역_코드_명",
    "주_부속_구분_코드",
    "주_부속_구분_코드_명",
    "구조_코드",
    "주_용도_코드",
    "지붕_코드",
    "층_구조_코드",
    "층_주_용도_코드",
    "층_주_용도_코드_명",
    "층_주_부속_구분_코드",
    "층_면적_제외_여부",
    "층_면적",
    "사용승인_일",
    CASE
        WHEN regexp_matches(사용승인_일, '^[0-9]{8}$')
        THEN CAST(LEFT(사용승인_일, 4) AS INTEGER)
        ELSE 0
    END AS 준공년도,
    CASE
        WHEN regexp_matches(사용승인_일, '^[0-9]{8}$')
        THEN 2024-CAST(LEFT(사용승인_일, 4) AS INTEGER)
        ELSE 100
    END AS 경과년수2024,
FROM
    joined_db
"""
result = con.sql(query)
print(result)

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬──────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │ 지역_코드_명 │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │ 층_주_용도_코드_명 │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │
│      varchar       │   varchar   │   varchar   │     varchar      │  varchar  │   varchar    │      varchar      │       varchar        │  varchar  │   varchar    │  varchar  │   varchar    │     varchar     │      varchar       │       varchar        │      varchar      │ double  │   varchar   │  int32   │    int32     │
├────────────────────┼─────────────┼─────────────┼──────────────────┼───────────┼──────────────┼───────────────────┼──────────────

In [7]:
bjd_df = pd.read_csv("../data/processed/code_bjd.csv")
bjd_df.head()

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


In [8]:
sgg_df = pd.read_csv("../data/processed/code_sgg.csv")
sgg_df.head()

,시군구코드,시군구명,폐지여부
0,11000,서울특별시,존재
1,11110,서울특별시 종로구,존재
2,11140,서울특별시 중구,존재
3,11170,서울특별시 용산구,존재
4,11200,서울특별시 성동구,존재


In [9]:
result2 = con.sql("""
    SELECT 
        r.*, 
        s."시군구명", 
    FROM result r
    LEFT JOIN sgg_df s
    ON r."시군구_코드" = s."시군구코드"
""")
result3 = con.sql("""
    SELECT 
        r.*, 
        d."법정동명", 
    FROM result2 r
    LEFT JOIN bjd_df d
    ON r."long_법정동_코드" = d."법정동코드"
""")
print(result3)

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬──────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬───────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬───────────────────┬─────────────────────────────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │ 지역_코드_명 │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │  층_주_용도_코드_명   │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │     시군구명      │            법정동명             │
│      varchar       │   varchar   │   varchar   │     varchar      │  varchar  │   varchar    │      varchar      │       varchar        │  varchar  │   varchar    │  varchar  │   varchar    │     varchar     │        varchar        │       varchar        │      varchar      │ double  │   varchar   │  int32   │    int32     │      varchar      │  

In [10]:
path_재조달구조내구연한 = Path("../data/재조달원가/재조달구조내구연한.csv")
path_재조달구조코드연계 = Path("../data/재조달원가/재조달구조코드연계.csv")
path_재조달용도구조원가내구연한 = Path(
    "../data/재조달원가/재조달용도구조원가내구연한.csv"
)
path_재조달용도원가 = Path("../data/재조달원가/재조달용도원가.csv")
path_재조달용도코드연계 = Path("../data/재조달원가/재조달용도코드연계.csv")

재조달구조내구연한 = pd.read_csv(
    path_재조달구조내구연한,
    encoding="cp949",
    dtype={"내구연한(중앙값)": float},
)
재조달구조코드연계 = pd.read_csv(
    path_재조달구조코드연계,
    encoding="cp949",
    dtype=str,
)
재조달용도구조원가내구연한 = pd.read_csv(
    path_재조달용도구조원가내구연한,
    encoding="cp949",
    dtype={
        "재조달원가 단가(원/㎡; 중앙값)": int,
        "내구연한(중앙값)": float,
    },
)
재조달용도원가 = pd.read_csv(
    path_재조달용도원가,
    encoding="cp949",
    dtype={"재조달원가 단가(원/㎡; 중앙값)": int},
)
재조달용도코드연계 = pd.read_csv(
    path_재조달용도코드연계,
    encoding="cp949",
    dtype=str,
)

print(재조달구조내구연한.head())
print(재조달구조코드연계.head())
print(재조달용도구조원가내구연한.head())
print(재조달용도원가.head())
print(재조달용도코드연계.head())

  재조달구조코드 재조달구조이름  내구연한(중앙값)
0     NaN      기타       45.0
1     C01      목조       40.0
2     C02     벽돌조       45.0
3     C03   치장벽돌조       45.0
4     C04     블록조       40.0
  구조_코드 재조달구조코드 재조달구조명
0   NaN     C00     기타
1     '     C00     기타
2     `     C00     기타
3    01     C00     기타
4    10     C02    벽돌조
  재조달용도코드 재조달용도명 재조달구조코드   재조달구조명  재조달원가 단가(원/㎡; 중앙값)  내구연한(중앙값)
0  A01B01   일반주택     C01       목조             1269000       40.0
1  A01B01   일반주택     C02      벽돌조             1446000       45.0
2  A01B01   일반주택     C03    치장벽돌조             1553500       45.0
3  A01B01   일반주택     C04      블록조             1120000       40.0
4  A01B01   일반주택     C05  철근콘크리트조             1521500       50.0
  재조달용도코드 재조달용도명  재조달원가 단가(원/㎡; 중앙값)
0  A01B01   일반주택             1387500
1  A01B02   고급주택             3562000
2  A01B03  통나무주택             2661000
3  A01B04  스틸하우스             2383000
4  A01B05  다가구주택             1874000
  층_주_용도_코드 층_주_용도_코드_명 재조달용도명 재조달용도코드
0     01000        단독주택   일반주택  A01B0

In [11]:
result4 = con.sql("""
    SELECT 
        r.*,
        u."재조달용도코드",
        u."재조달용도명",
        s."재조달구조코드",
        s."재조달구조명",
    FROM result3 r
    LEFT JOIN 재조달용도코드연계 u
    ON r."층_주_용도_코드" = u."층_주_용도_코드"
    LEFT JOIN 재조달구조코드연계 s
    ON r."층_구조_코드" = s."구조_코드"
""")
print(result4)

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬──────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬─────────────────┬─────────────────────────────────┬────────────────┬──────────────┬────────────────┬────────────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │ 지역_코드_명 │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │ 층_주_용도_코드_명 │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │    시군구명     │            법정동명             │ 재조달용도코드 │ 재조달용도명 │ 재조달구조코드 │  재조달구조명  │
│      varchar       │   varchar   │   varchar   │     varchar      │  varchar  │   varchar    │      varchar      │       varchar        │  varchar  │   varchar    │  varchar  │   varchar    │     varchar     │      varchar       │       varchar        

In [12]:
result5 = con.sql("""
    SELECT 
        r.*,
        COALESCE(
            b."재조달원가 단가(원/㎡; 중앙값)", 
            u."재조달원가 단가(원/㎡; 중앙값)")     AS "재조달_단가",
        COALESCE(
            b."내구연한(중앙값)", 
            s."내구연한(중앙값)", 
            45.0)                                   AS "내구연한",
    FROM result4 r
    LEFT JOIN 재조달용도구조원가내구연한 b
    ON r."재조달용도코드" = b."재조달용도코드"
        AND r."재조달구조코드" = b."재조달구조코드"
    LEFT JOIN 재조달용도원가 u
    ON r."재조달용도코드" = u."재조달용도코드"
    LEFT JOIN 재조달구조내구연한 s
    ON r."재조달구조코드" = s."재조달구조코드"
""")
print(result5)

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬──────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬───────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬─────────────────┬─────────────────────────────────┬────────────────┬──────────────┬────────────────┬────────────────┬─────────────┬──────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │ 지역_코드_명 │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │  층_주_용도_코드_명   │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │    시군구명     │            법정동명             │ 재조달용도코드 │ 재조달용도명 │ 재조달구조코드 │  재조달구조명  │ 재조달_단가 │ 내구연한 │
│      varchar       │   varchar   │   varchar   │     varchar      │  varchar  │   varchar    │      varchar      │       varchar        │  varchar  │   varchar    │  varchar  │   varchar    │     varchar  

In [13]:
result6 = con.sql("""
    SELECT 
        *,
        GREATEST(
            (내구연한 - 경과년수2024) / NULLIF(내구연한, 0),
            0.1
        ) AS 잔존비율_최소값보정,        
    FROM result5
""")
result7 = con.sql("""
    SELECT 
        *,
        내구연한*잔존비율_최소값보정 AS 잔존연한,        
    FROM result6
""")
print(result7)

┌────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬──────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬───────────────────┬─────────────────────────────────┬────────────────┬──────────────────┬────────────────┬────────────────┬─────────────┬──────────┬─────────────────────┬────────────────────┐
│ 관리_건축물대장_PK │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │ 지역_코드_명 │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │ 층_주_용도_코드_명 │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │     시군구명      │            법정동명             │ 재조달용도코드 │   재조달용도명   │ 재조달구조코드 │  재조달구조명  │ 재조달_단가 │ 내구연한 │ 잔존비율_최소값보정 │      잔존연한      │
│      varchar       │   varchar   │   varchar   │     varchar      │  varchar  │   varchar    │      varchar      │       varch

In [14]:
result8 = con.sql("""
    SELECT 
        *,
        CAST(COALESCE(층_면적*재조달_단가,0) AS INT64) AS 재조달원가,       
        CAST(COALESCE(재조달원가*잔존비율_최소값보정, 0) AS INT64) AS 잔존재조달원가,       
    FROM result7
""")
print(result8)

┌────────────────────────┬─────────────┬─────────────┬──────────────────┬───────────┬───────────────────┬───────────────────┬──────────────────────┬───────────┬──────────────┬───────────┬──────────────┬─────────────────┬────────────────────┬──────────────────────┬───────────────────┬─────────┬─────────────┬──────────┬──────────────┬───────────────────┬─────────────────────────────────┬────────────────┬────────────────────────────┬────────────────┬────────────────────┬─────────────┬──────────┬─────────────────────┬────────────────────┬────────────┬────────────────┐
│   관리_건축물대장_PK   │ 시군구_코드 │ 법정동_코드 │ long_법정동_코드 │ 지역_코드 │   지역_코드_명    │ 주_부속_구분_코드 │ 주_부속_구분_코드_명 │ 구조_코드 │ 주_용도_코드 │ 지붕_코드 │ 층_구조_코드 │ 층_주_용도_코드 │ 층_주_용도_코드_명 │ 층_주_부속_구분_코드 │ 층_면적_제외_여부 │ 층_면적 │ 사용승인_일 │ 준공년도 │ 경과년수2024 │     시군구명      │            법정동명             │ 재조달용도코드 │        재조달용도명        │ 재조달구조코드 │    재조달구조명    │ 재조달_단가 │ 내구연한 │ 잔존비율_최소값보정 │      잔존연한      │ 재조달원가 │ 잔존재조달원가 │
│        varchar         │   varch

In [15]:
count = con.execute("SELECT COUNT(*) FROM result8").fetchone()[0]  # type: ignore
print(count)
sum_area = con.execute("SELECT sum(층_면적) FROM result8").fetchone()[0]  # type: ignore
print(f"{sum_area:,.0f}")
sum_재조달원가 = con.execute("SELECT sum(재조달원가) FROM result8").fetchone()[0]  # type: ignore
print(f"{sum_재조달원가:,.0f}")
sum_잔존재조달원가 = con.execute("SELECT sum(잔존재조달원가) FROM result8").fetchone()[
    0
]  # type: ignore
print(f"{sum_잔존재조달원가:,.0f}")

19614969
4,372,971,283
6,479,141,282,005,889
3,898,904,904,089,070


In [16]:
# Save result as a parquet file
path_to = data_root / ("econvalue_재조달원가.parquet")
query = f"COPY result8 TO '{path_to.as_posix()}' (FORMAT parquet);"
con.execute(query)

In [17]:
pd.read_parquet(path_to).head()

,관리_건축물대장_PK,시군구_코드,법정동_코드,long_법정동_코드,지역_코드,지역_코드_명,주_부속_구분_코드,주_부속_구분_코드_명,구조_코드,주_용도_코드,...,재조달용도코드,재조달용도명,재조달구조코드,재조달구조명,재조달_단가,내구연한,잔존비율_최소값보정,잔존연한,재조달원가,잔존재조달원가
0,1110126357,41360,25629,4136025629,Z200,준농림지역,0,주건축물,21,02000,...,A02B02,연립주택,C05,철근콘크리트조,2082000.0,50.0,0.440,22.0,509298840,224091490
1,1110126357,41360,25629,4136025629,Z200,준농림지역,0,주건축물,21,02000,...,A02B02,연립주택,C05,철근콘크리트조,2082000.0,50.0,0.440,22.0,509298840,224091490
2,1110126443,41360,26221,4136026221,None,None,0,주건축물,12,01000,...,A01B01,일반주택,C04,블록조,1120000.0,40.0,0.100,4.0,48641600,4864160
3,1110126452,41360,34023,4136034023,0300,농림지역,0,주건축물,32,01000,...,A01B01,일반주택,C06,철골조,2131000.0,40.0,0.275,11.0,76716000,21096900
4,1110126462,41360,34026,4136034026,None,None,0,주건축물,21,01000,...,A01B01,일반주택,C05,철근콘크리트조,1521500.0,50.0,0.460,23.0,128445030,59084714
